In [ ]:
### Install dependencies

from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results
import requests
import os
import glob
import json
import itertools
import pandas as pd
import numpy as np
from time import sleep
import pprint


In [ ]:
## Create lists of time periods per day per month from March 2019 to August 2020
### Sample 10000 a day from March-August
months = ['03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 
          '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 
          '01', '02', '03', '04', '05']

years = [*itertools.repeat('2019',10),
         *itertools.repeat('2020',12), 
         *itertools.repeat('2021',5)]

periods_morning = []

## Create periods from random dates
for (month, year) in zip(months, years):
    duration = [] 
    if year in ['2019', '2021']:
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 29)]
    elif year == "2020":
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 30)]
    for date in dates:
        since = f"{year}-{month}-{date}T13:00:00Z" #9am ET is 4 hours behind UTC
        until = f"{year}-{month}-{date}T16:59:59Z" #end at 1pm ET (13:00), still four hours behind
        duration.append([since, until])
    for item in duration:
        periods_morning.append(item)

# len(periods_morning)

In [ ]:
periods_afternoon = []

## Create periods from random dates
for (month, year) in zip(months, years):
    duration = [] 
    if year in ['2019', '2021']:
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 29)]
    elif year == "2020":
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 30)]
    for date in dates:
        since = f"{year}-{month}-{date}T17:00:00Z" #1pm ET (13:00) is 4 hours behind UTC
        until = f"{year}-{month}-{date}T19:59:59Z" #end at 4pm ET (16:00), which is 4 hrs behind UTC
        duration.append([since, until])
    for item in duration:
        periods_afternoon.append(item)

# len(periods_afternoon)

In [ ]:
periods_evening = []

## Create periods from random dates
for (month, year) in zip(months, years):
    duration = [] 
    if year in ['2019', '2021']:
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 29)]
    elif year == "2020":
        #sample for months with 30 days
        if month in ['04', '06', '09', '11']:
            dates = ["%.2d" % i for i in range(1,31)]
        #sample for months with 31 days
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            dates = ["%.2d" % i for i in range(1,32)]
        #sample for Feb
        elif month == '02':
            dates = ["%.2d" % i for i in range(1, 30)]
    for date in dates:
        since = f"{year}-{month}-{date}T20:00:00Z" #4pm ET (16:00) is 4 hours behind UTC
        until = f"{year}-{month}-{date}T23:59:59Z" #end at 8pm ET (20:00), which is 4 hrs behind UTC
        duration.append([since, until])
    for item in duration:
        periods_evening.append(item)

# len(periods_evening)

In [ ]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = "AAAAAAAAAAAAAAAAAAAAAIgNEQEAAAAAocbZoug47z3mnfJPrk8qC%2FzjsD8%3DTdX2KB5ICEye31VGIKi0CM8sqR1RDWBP3ugt4gOE6iL3ll5kz3"
search_url = "https://api.twitter.com/2/tweets/search/all"

# Define bearer authentication function
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r

# Define connect_to_endpoint function
def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

# Define number_of_tweets function
def number_of_tweets(json_response):
    number = len(json_response["data"])
    return number

# Define main function which will pull tweets
def main(number_of_tweets_wanted, start_time):
    
    folder = glob.glob('*.csv')
    start_time_filename = start_time.replace(":", "-")
    
    if f"response_{start_time_filename}.csv" not in folder:
        json_file = {}
        json_file['data'] = []

        json_response = connect_to_endpoint(search_url, query_params)
        sleep(2)
        
        if "data" in json_response.keys():
            if "includes" in json_response.keys():
                for geo in json_response["includes"]["places"]:
                    for data in json_response["data"]:
                        if "geo" in data.keys():
                            if data['geo']['place_id'] == geo['id']:
                                data['geo']['full_name'] = geo['full_name']
                                data['geo']['bbox'] = geo['geo']['bbox']
                                data['geo']['place_type'] = geo['place_type']

            for item in json_response['data']:
                json_file['data'].append(item)

            ## Set up loop
            if len(json_file['data']) < number_of_tweets_wanted:
                if 'next_token' in json_response['meta'].keys():
                    query_params['pagination_token'] = json_response['meta']['next_token']
                    json_response = connect_to_endpoint(search_url, query_params)
                    sleep(2)
                    
                    if "data" in json_response.keys():
                        if "includes" in json_response.keys():
                            for geo in json_response["includes"]["places"]:
                                for data in json_response["data"]:
                                    if "geo" in data.keys():
                                        if data['geo']['place_id'] == geo['id']:
                                            data['geo']['full_name'] = geo['full_name']
                                            data['geo']['bbox'] = geo['geo']['bbox']
                                            data['geo']['place_type'] = geo['place_type']
                                        else:
                                            pass
                                    else:
                                        pass
                        else:
                            pass
                        for item in json_response['data']:
                            json_file['data'].append(item)
                    else:
                        pass
                else:
                    pass
            else:
                pass
        else:
            pass

        ## Convert to df       
        df = pd.DataFrame.from_dict(pd.json_normalize(json_file['data']), orient='columns')
        df.to_csv(f"response_{start_time_filename}.csv", index=False)
        print(json.dumps(json_response, indent=4, sort_keys=True))
    else:
        pass


In [ ]:
######### Pull tweets - COVID

# for period in periods_evening: ## 360, 720
#     # Set query
#     query = "coronavirus OR COVID-19 OR Wuhan%20virus OR China%20virus -RT -is:retweet place_country:US lang:en"

#     #Set max number of tweets wanted
#     number_of_tweets_wanted = 5000

#     # Consolidate into params dict
#     query_params = {'query': query,
#                     'max_results': 100, #this is per page, irrelevant to our aims, 100 is max
#                     'expansions': 'geo.place_id',
#                     'place.fields': "geo,place_type",
#                     'tweet.fields': 'author_id'}
    
#     # Define period
#     start_time = period[0]
#     end_time = period[1]
    
#     # Update query
#     query_params['start_time'] = start_time
#     query_params['end_time'] = end_time
#     print(query_params)
    
#     # Pull
#     main(number_of_tweets_wanted, start_time)
    

In [ ]:
########## Pull tweets - Asians

# for period in periods_afternoon: 
#     # Set query
#     query = "Asian OR Asians OR Chinese place_country:US -RT -is:retweet lang:en"

#     #Set max number of tweets wanted
#     number_of_tweets_wanted = 5000

#     # Consolidate into params dict
#     query_params = {'query': query,
#                     'max_results': 100, #this is per page, irrelevant to our aims, 100 is max
#                     'expansions': 'geo.place_id',
#                     'place.fields': "geo,place_type",
#                     'tweet.fields': 'author_id'}
#     # Select periods
#     start_time = period[0]
#     end_time = period[1]
    
#     # Update query
#     query_params['start_time'] = start_time
#     query_params['end_time'] = end_time
#     print(query_params)
    
#     # Pull
#     main(number_of_tweets_wanted, start_time)


In [ ]:
########## Pull tweets - Asians or COVID

# for period in periods_evening:
#     # Set query
#     query = "(Asian OR Chinese) OR (coronavirus OR COVID-19 OR Wuhan%20virus OR China%20virus) place_country:US -RT -is:retweet lang:en"

#     #Set max number of tweets wanted
#     number_of_tweets_wanted = 5000

#     # Consolidate into params dict
#     query_params = {'query': query,
#                     'max_results': 100, #this is per page, irrelevant to our aims, 100 is max
#                     'expansions': 'geo.place_id',
#                     'place.fields': "geo,place_type",
#                     'tweet.fields': 'author_id'}
#     # Select periods
#     start_time = period[0]
#     end_time = period[1]
    
#     # Update query
#     query_params['start_time'] = start_time
#     query_params['end_time'] = end_time
#     print(query_params)
    
#     # Pull
#     main(number_of_tweets_wanted, start_time)


In [ ]:
########## Pull tweets - Asians & COVID

# for period in periods_morning:
#     # Set query
#     query = "(Asian OR Chinese) AND (coronavirus OR COVID-19 OR Wuhan%20virus OR China%20virus) place_country:US -RT -is:retweet lang:en"

#     #Set max number of tweets wanted
#     number_of_tweets_wanted = 5000

#     # Consolidate into params dict
#     query_params = {'query': query,
#                     'max_results': 100, #this is per page, irrelevant to our aims, 100 is max
#                     'expansions': 'geo.place_id',
#                     'place.fields': "geo,place_type",
#                     'tweet.fields': 'author_id'}
#     # Select periods
#     start_time = period[0]
#     end_time = period[1]
    
#     # Update query
#     query_params['start_time'] = start_time
#     query_params['end_time'] = end_time
#     print(query_params)
    
#     # Pull
#     main(number_of_tweets_wanted, start_time)
#     sleep(1)

In [ ]:
########## Pull for 31st of all months

# for period in periods_31st: #insert periods in brackets (e.g., [0:2] limits the list to first 2 elements, element 0 and element 1 (2 not included)) if you don't want to pull for all periods
#     # Set query
#     query = "(Asian OR Chinese) OR (coronavirus OR COVID-19 OR Wuhan%20virus OR China%20virus) place_country:US -RT -is:retweet lang:en"

#     #Set max number of tweets wanted
#     number_of_tweets_wanted = 5000

#     # Consolidate into params dict
#     query_params = {'query': query,
#                     'max_results': 100, #this is per page, irrelevant to our aims, 100 is max
#                     'expansions': 'geo.place_id',
#                     'place.fields': "geo,place_type",
#                     'tweet.fields': 'author_id'}
#     # Select periods
#     start_time = period[0]
#     end_time = period[1]
    
#     # Update query
#     query_params['start_time'] = start_time
#     query_params['end_time'] = end_time
#     print(query_params)
    
#     # Pull
#     main(number_of_tweets_wanted, start_time)
#     sleep(1)

In [ ]:
periods_evening[-1]